# Imports

In [1]:
from random import choice
from random import randint
from pandas import DataFrame as Table
from itertools import permutations

# Itemset

In [2]:
ITEMS = set(['l1','l2','l3','l4','l5'])

In [3]:
ITEMS

{'l1', 'l2', 'l3', 'l4', 'l5'}

# Generating a random transaction dataset

In [4]:
def generate_random_transactional_dataset(items,min_samples=1,max_samples=10,entries=20):
    
    Td_list = [set([choice(list(items)) for _ in range(randint(min_samples,max_samples))]) for _ in range(entries)]
    Td_dict = {}
    
    for s in Td_list:
        if frozenset(s) not in Td_dict.keys():
            Td_dict[frozenset(s)] = 1
        else:
            Td_dict[frozenset(s)] += 1
            
    return Td_list,Td_dict

Td_list,Td_dict = generate_random_transactional_dataset(ITEMS)

In [5]:
Td_list

[{'l2', 'l3'},
 {'l2', 'l3', 'l4', 'l5'},
 {'l2'},
 {'l1', 'l2', 'l3', 'l4'},
 {'l1', 'l2', 'l3', 'l4', 'l5'},
 {'l2', 'l4', 'l5'},
 {'l1', 'l4'},
 {'l1', 'l2', 'l3', 'l5'},
 {'l1', 'l2', 'l3', 'l5'},
 {'l1', 'l3', 'l5'},
 {'l1', 'l2', 'l4', 'l5'},
 {'l1', 'l4'},
 {'l5'},
 {'l2', 'l3', 'l4', 'l5'},
 {'l1'},
 {'l1', 'l3'},
 {'l1', 'l3', 'l5'},
 {'l2', 'l3', 'l5'},
 {'l1', 'l2', 'l3', 'l4', 'l5'},
 {'l1', 'l3', 'l4', 'l5'}]

In [6]:
Td_dict

{frozenset({'l2', 'l3'}): 1,
 frozenset({'l2', 'l3', 'l4', 'l5'}): 2,
 frozenset({'l2'}): 1,
 frozenset({'l1', 'l2', 'l3', 'l4'}): 1,
 frozenset({'l1', 'l2', 'l3', 'l4', 'l5'}): 2,
 frozenset({'l2', 'l4', 'l5'}): 1,
 frozenset({'l1', 'l4'}): 2,
 frozenset({'l1', 'l2', 'l3', 'l5'}): 2,
 frozenset({'l1', 'l3', 'l5'}): 2,
 frozenset({'l1', 'l2', 'l4', 'l5'}): 1,
 frozenset({'l5'}): 1,
 frozenset({'l1'}): 1,
 frozenset({'l1', 'l3'}): 1,
 frozenset({'l2', 'l3', 'l5'}): 1,
 frozenset({'l1', 'l3', 'l4', 'l5'}): 1}

# One hot encoding

In [7]:
def one_hot_encoding(items, Td_list):
    items = list(items)
    items.sort()
    return Table([{ it : it in tr  for it in items} for tr in Td_list])

Td_table = one_hot_encoding(ITEMS,Td_list)

In [8]:
Td_table

,l1,l2,l3,l4,l5
0,False,True,True,False,False
1,False,True,True,True,True
2,False,True,False,False,False
3,True,True,True,True,False
4,True,True,True,True,True
5,False,True,False,True,True
6,True,False,False,True,False
7,True,True,True,False,True
8,True,True,True,False,True
9,True,False,True,False,True


# Support

In [9]:
def support(Td_table,tr):
    tr = list(it for it in tr)
    filtered = Td_table
    for it in tr:
        filtered = filtered.loc[lambda x : x[it]]
    return filtered[tr].shape[0] / Td_table.shape[0]

# A priori algorithm

In [10]:
def next_candidates(Fi_k,items):
    
    # Generate all possibile combinations of each element of Fi_k with another one of items
    unique_combinations = []
    for tr in Fi_k:
        for it in items.difference(tr):
            combination = tr.copy()
            combination.add(it)
            if combination not in unique_combinations:
                unique_combinations.append(combination)

    # Prune candidates
    candidates = []
    
    for combination in unique_combinations:

        discard_transaction = False
        
        # Generate all possible proper subsets
        for it in combination:
            max_proper_subset = combination.copy()
            max_proper_subset.discard(it)

            # Consequence 2 : k-size candidate
            if max_proper_subset not in Fi_k:
                discard_transaction = True
                break
        
        if not discard_transaction:
            candidates.append(combination)

    return candidates

def Apriori(Td_table,epsilon,items):

    Fi = set()
    candidates = [set([it]) for it in items]
    
    while True:
        Fi_k = [tr for tr in candidates if support(Td_table,tr)>epsilon]
        Fi = Fi.union(set([frozenset(tr) for tr in Fi_k]))
        candidates = next_candidates(Fi_k,items)
        if not candidates:
            break
    return Fi

Fi = Apriori(Td_table,0.2,ITEMS)
Fi

{frozenset({'l5'}),
 frozenset({'l3'}),
 frozenset({'l2', 'l3'}),
 frozenset({'l1'}),
 frozenset({'l1', 'l3'}),
 frozenset({'l1', 'l2', 'l3'}),
 frozenset({'l1', 'l2', 'l5'}),
 frozenset({'l3', 'l5'}),
 frozenset({'l4'}),
 frozenset({'l4', 'l5'}),
 frozenset({'l2'}),
 frozenset({'l2', 'l4'}),
 frozenset({'l2', 'l4', 'l5'}),
 frozenset({'l3', 'l4'}),
 frozenset({'l2', 'l3', 'l5'}),
 frozenset({'l2', 'l3', 'l4'}),
 frozenset({'l3', 'l4', 'l5'}),
 frozenset({'l2', 'l5'}),
 frozenset({'l1', 'l2'}),
 frozenset({'l1', 'l5'}),
 frozenset({'l1', 'l3', 'l5'}),
 frozenset({'l1', 'l4'})}